In [ ]:
import os
from obspy import read
import seisbench.models as sbm

# Load the model
model = sbm.DeepDenoiser.from_pretrained("original")

# Define the source and target directories
source_directory = '/Users/marcgarcia/Research/Picking/PhaseNet/Data/Raw'
target_directory = '/Users/marcgarcia/Research/Picking/PhaseNet/Data/Denoised/'

# Ensure the target directory exists
os.makedirs(target_directory, exist_ok=True)

# Get all miniseed files
mseed_files = [f for f in os.listdir(source_directory) if f.endswith('.mseed')]

# Sort files by their names
mseed_files.sort()

# Loop through each sorted file
for filename in mseed_files:
    file_path = os.path.join(source_directory, filename)
    
    # Read the stream
    stream = read(file_path)

    # Apply the model to denoise
    denoised = model.annotate(stream)

    # Edit Traces
    for trace in denoised:
        if 'DeepDenoiser_' in trace.stats.channel:
            trace.stats.channel = trace.stats.channel.replace('DeepDenoiser_', '')
    
    target_file_path = os.path.join(target_directory, filename)
    
    # Write denoised signals to the target directory
    denoised.write(target_file_path, format='MSEED')

    print(f'Processed and saved {filename}')


In [2]:
import os
import torch
from obspy import read
import seisbench.models as sbm
from concurrent.futures import ThreadPoolExecutor, as_completed

# Set device to GPU if available, else CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu
Processed and saved 2017-09-06T01-00-00.mseed
Processed and saved 2017-09-06T00-00-00.mseed
Processed and saved 2017-09-06T04-00-00.mseed
Processed and saved 2017-09-06T06-00-00.mseed
Processed and saved 2017-09-06T03-00-00.mseed
Processed and saved 2017-09-06T02-00-00.mseed
Processed and saved 2017-09-06T05-00-00.mseed
Processed and saved 2017-09-06T08-00-00.mseed
Processed and saved 2017-09-06T07-00-00.mseed
Processed and saved 2017-09-06T09-00-00.mseed
Processed and saved 2017-09-06T10-00-00.mseed
Processed and saved 2017-09-06T11-00-00.mseed
Processed and saved 2017-09-06T12-00-00.mseed
Processed and saved 2017-09-06T13-00-00.mseed


In [1]:
import os
import torch
from obspy import read
import seisbench.models as sbm
from concurrent.futures import ThreadPoolExecutor, as_completed

# Set the number of threads for PyTorch to n-1 (where n is the total number of performance cores on your CPU)
torch.set_num_threads(7)  # Adjust this based on your CPU's core count if different

# Function to process each file
def process_file(filename, source_directory, target_directory, model):
    file_path = os.path.join(source_directory, filename)
    
    # Read the stream
    stream = read(file_path)

    # Apply the model to denoise
    denoised = model.annotate(stream)

    # Edit Traces
    for trace in denoised:
        if 'DeepDenoiser_' in trace.stats.channel:
            trace.stats.channel = trace.stats.channel.replace('DeepDenoiser_', '')
    
    target_file_path = os.path.join(target_directory, filename)
    
    # Write denoised signals to the target directory
    denoised.write(target_file_path, format='MSEED')
    
    print(f'Processed and saved {filename}')

# Load the model
model = sbm.DeepDenoiser.from_pretrained("original")

# Define the source and target directories
source_directory = '/Users/marcgarcia/Research/Picking/PhaseNet/Data/Raw'
target_directory = '/Users/marcgarcia/Research/Picking/PhaseNet/Data/Denoised/'

# Ensure the target directory exists
os.makedirs(target_directory, exist_ok=True)

# Get all miniseed files
mseed_files = [f for f in os.listdir(source_directory) if f.endswith('.mseed')]

# Sort files by their names
mseed_files.sort()

# Number of threads to use for parallel processing - adjust based on experimentation
num_threads = 7

# Using ThreadPoolExecutor to process files in parallel
with ThreadPoolExecutor(max_workers=num_threads) as executor:
    # Submit all the tasks to the executor
    future_to_file = {executor.submit(process_file, filename, source_directory, target_directory, model): filename for filename in mseed_files}
    
    # Process the results as they complete
    for future in as_completed(future_to_file):
        filename = future_to_file[future]
        try:
            # Retrieve result (if any)
            result = future.result()
        except Exception as exc:
            print(f'{filename} generated an exception: {exc}')



KeyboardInterrupt: 

In [ ]:
# Read stream
stream = read('/Users/marcgarcia/Research/Picking/PhaseNet/Data/Denoised/2017-09-06T00-00-00.mseed')
print(stream.__str__(extended=True))

In [ ]:
from obspy import read

stream = read("/Users/marcgarcia/Research/Picking/PhaseNet/Data/Denoised/2017-09-06T00-00-00.mseed")
for trace in stream:
    print(trace.stats)


In [ ]:
from obspy import read

stream = read("/Users/marcgarcia/Research/Picking/PhaseNet/Data/Raw/2017-09-06T00-00-00.mseed")
print(stream)
for trace in stream:
    print(trace.stats)